In [335]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

In [377]:
address = 'New York city, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [378]:
CLIENT_ID = '5KEBPGWXTZEYIZVNHXFIULMZ2AYL3B5EBTXFBZ3CQBLGUOIN' # your Foursquare ID
CLIENT_SECRET = '5FIOYWNNOL4WUCPEE02AQEOXEY1ZRT3E1B5LPQCR1GMMAGXU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5KEBPGWXTZEYIZVNHXFIULMZ2AYL3B5EBTXFBZ3CQBLGUOIN
CLIENT_SECRET:5FIOYWNNOL4WUCPEE02AQEOXEY1ZRT3E1B5LPQCR1GMMAGXU


In [385]:
# type your answer here
LIMIT = 500
radius = 100000
categoryId='4bf58dd8d48988d113951735'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&limit={}&radius={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    categoryId,
    LIMIT,
    radius
)
url 



'https://api.foursquare.com/v2/venues/explore?&client_id=5KEBPGWXTZEYIZVNHXFIULMZ2AYL3B5EBTXFBZ3CQBLGUOIN&client_secret=5FIOYWNNOL4WUCPEE02AQEOXEY1ZRT3E1B5LPQCR1GMMAGXU&v=20180605&ll=40.7127281,-74.0060152&categoryId=4bf58dd8d48988d113951735&limit=500&radius=100000'

In [386]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4e3b08ad1ab4001b3cde27'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'New York',
  'headerFullLocation': 'New York',
  'headerLocationGranularity': 'city',
  'query': 'gas station',
  'totalResults': 402,
  'suggestedBounds': {'ne': {'lat': 41.6127290000009,
    'lng': -72.8208793268391},
   'sw': {'lat': 39.8127271999991, 'lng': -75.19115107316088}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5cd47b5c66fc65002cc66239',
       'name': 'Costco Gasoline',
       'location': {'lat': 40.672759,
        'lng': -74.105526,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.672759,
          'lng': -74.105526}],
        'dis

In [387]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [388]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues=nearby_venues[nearby_venues["categories"]=="Gas Station"]
nearby_venues

,name,categories,lat,lng
0,Costco Gasoline,Gas Station,40.672759,-74.105526
1,Costco Gasoline,Gas Station,40.831932,-74.138382
2,BP,Gas Station,40.730945,-73.951550
3,Speedway,Gas Station,40.731936,-74.039191
4,Costco Gasoline,Gas Station,40.861959,-74.063968
5,Mobil,Gas Station,40.679297,-73.953584
7,BP,Gas Station,40.719597,-73.891197
8,Fuel4 Gas & Diesel,Gas Station,40.731900,-74.040842
9,Costco Gasoline,Gas Station,40.891060,-74.251221
10,Wawa,Gas Station,40.862729,-74.036430


In [389]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

81 venues were returned by Foursquare.


In [390]:
# create map and display it
ny_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# display the map of San Francisco
ny_map

In [391]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(nearby_venues.lat, nearby_venues.lng):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
ny_map.add_child(incidents)

In [392]:
data = pd.read_csv(r"C:\Users\Aaftaab\Desktop\Zeenat\Coursera_Capstone\Traffic_Volume.csv") 
data.head()

,ID,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,Total
0,2,133472,BRUCKNER EXPRESSWAY,LONGFELLOW AVENUE,BRYANT AVENUE,NB,11/1/2016,423,280,201,177,197,268,505,871,841,644,564,618,678,676,650,507,663,616,711,836,700,621,531,452,13230
1,2,239605,BRUCKNER EXPRESSWAY,LONGFELLOW AVENUE,BRYANT AVENUE,SB,11/1/2016,246,162,111,166,340,"1,249","1,830","1,712","1,330","1,219","1,158","1,063",908,"1,024","1,042",952,880,892,856,724,572,555,515,396,19902
2,3,135514,MAJOR DEEGAN EXPRESSWAY,MOSHOLU PARKWAY,VAN CORTLANDT PARK SOUTH,SB,11/1/2016,957,594,548,589,"1,138","3,089","3,272","2,393","2,290","2,932","2,612","2,836","2,879","2,869","3,105","3,412","3,505","3,887","3,649","2,859","2,336","2,119","1,975","1,527",57372
3,3,135515,MAJOR DEEGAN EXPRESSWAY,MOSHOLU PARKWAY,VAN CORTLANDT PARK SOUTH,NB,11/1/2016,"1,274",747,542,539,629,"1,202","2,466","3,259","3,111","3,180","3,129","3,679","3,054","3,056","3,031","4,322","4,506","3,772","4,089","3,701","3,074","2,565","2,416","1,929",63272
4,5,139037,BRUCKNER EXPRESSWAY,OLMSTEAD AVENUE,CASTLE HILL AVENUE,WB,11/1/2016,739,547,489,545,"1,076","2,833","3,071","2,726","3,285","3,507","3,597","3,379","3,440","3,611","3,656","3,723","3,480","3,541","3,388","2,786","2,210","1,885","1,688","1,266",60468


In [393]:
data["Name_id"] = data["Roadway Name"]+"-" + data["Direction"]
data['Name_id'] = data['Name_id'].str.upper()
data.head()

,ID,Segment ID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,3:00-4:00AM,4:00-5:00AM,5:00-6:00AM,6:00-7:00AM,7:00-8:00AM,8:00-9:00AM,9:00-10:00AM,10:00-11:00AM,11:00-12:00PM,12:00-1:00PM,1:00-2:00PM,2:00-3:00PM,3:00-4:00PM,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,Total,Name_id
0,2,133472,BRUCKNER EXPRESSWAY,LONGFELLOW AVENUE,BRYANT AVENUE,NB,11/1/2016,423,280,201,177,197,268,505,871,841,644,564,618,678,676,650,507,663,616,711,836,700,621,531,452,13230,BRUCKNER EXPRESSWAY-NB
1,2,239605,BRUCKNER EXPRESSWAY,LONGFELLOW AVENUE,BRYANT AVENUE,SB,11/1/2016,246,162,111,166,340,"1,249","1,830","1,712","1,330","1,219","1,158","1,063",908,"1,024","1,042",952,880,892,856,724,572,555,515,396,19902,BRUCKNER EXPRESSWAY-SB
2,3,135514,MAJOR DEEGAN EXPRESSWAY,MOSHOLU PARKWAY,VAN CORTLANDT PARK SOUTH,SB,11/1/2016,957,594,548,589,"1,138","3,089","3,272","2,393","2,290","2,932","2,612","2,836","2,879","2,869","3,105","3,412","3,505","3,887","3,649","2,859","2,336","2,119","1,975","1,527",57372,MAJOR DEEGAN EXPRESSWAY-SB
3,3,135515,MAJOR DEEGAN EXPRESSWAY,MOSHOLU PARKWAY,VAN CORTLANDT PARK SOUTH,NB,11/1/2016,"1,274",747,542,539,629,"1,202","2,466","3,259","3,111","3,180","3,129","3,679","3,054","3,056","3,031","4,322","4,506","3,772","4,089","3,701","3,074","2,565","2,416","1,929",63272,MAJOR DEEGAN EXPRESSWAY-NB
4,5,139037,BRUCKNER EXPRESSWAY,OLMSTEAD AVENUE,CASTLE HILL AVENUE,WB,11/1/2016,739,547,489,545,"1,076","2,833","3,071","2,726","3,285","3,507","3,597","3,379","3,440","3,611","3,656","3,723","3,480","3,541","3,388","2,786","2,210","1,885","1,688","1,266",60468,BRUCKNER EXPRESSWAY-WB


In [394]:
new_df=data.groupby('Name_id',as_index=False).agg({"Total": "sum"})
new_df.head()

,Name_id,Total
0,1 AVENUE-NB,1106668
1,10 AVENUE-NB,843325
2,103 STREET-NB,28951
3,108 STREET-NB,84009
4,108 STREET-SB,138291


In [395]:
new_df=new_df.sort_values(by=['Total'], ascending=False)
new_df=new_df.head(20)
new_df

,Name_id,Total
222,CROSS BRONX EXPRESSWAY-EB,2651260
223,CROSS BRONX EXPRESSWAY-WB,2167675
135,BELT PARKWAY-EB,2029041
138,BELT PARKWAY-WB,2026221
308,GRAND CENTRAL PARKWAY-SB,1592929
492,STATEN ISLAND EXPRESSWAY-EB,1509227
493,STATEN ISLAND EXPRESSWAY-WB,1429728
25,2 AVENUE-SB,1155153
0,1 AVENUE-NB,1106668
369,MADISON AVENUE-NB,1045637


In [396]:
new = new_df["Name_id"].str.split("-", n = 1, expand = True) 
new_df["Rd"]=new[0]
new_df["Direction"]=new[1]
new_df

,Name_id,Total,Rd,Direction
222,CROSS BRONX EXPRESSWAY-EB,2651260,CROSS BRONX EXPRESSWAY,EB
223,CROSS BRONX EXPRESSWAY-WB,2167675,CROSS BRONX EXPRESSWAY,WB
135,BELT PARKWAY-EB,2029041,BELT PARKWAY,EB
138,BELT PARKWAY-WB,2026221,BELT PARKWAY,WB
308,GRAND CENTRAL PARKWAY-SB,1592929,GRAND CENTRAL PARKWAY,SB
492,STATEN ISLAND EXPRESSWAY-EB,1509227,STATEN ISLAND EXPRESSWAY,EB
493,STATEN ISLAND EXPRESSWAY-WB,1429728,STATEN ISLAND EXPRESSWAY,WB
25,2 AVENUE-SB,1155153,2 AVENUE,SB
0,1 AVENUE-NB,1106668,1 AVENUE,NB
369,MADISON AVENUE-NB,1045637,MADISON AVENUE,NB


In [397]:
def locate(road):
    location = None
    while(location is None):
        address = ', New York City,NY,USA'
        address=road+address
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        loc=str(latitude)+'-'+str(longitude)
        return loc

In [398]:
new_df["Lat"]=new_df["Rd"].apply(locate)

In [399]:
new = new_df["Lat"].str.split("-", n = 1, expand = True) 
new_df["Lat"]=new[0]
new_df["Long"]=new[1]
new_df

,Name_id,Total,Rd,Direction,Lat,Long
222,CROSS BRONX EXPRESSWAY-EB,2651260,CROSS BRONX EXPRESSWAY,EB,40.8280764,-73.8414339
223,CROSS BRONX EXPRESSWAY-WB,2167675,CROSS BRONX EXPRESSWAY,WB,40.8280764,-73.8414339
135,BELT PARKWAY-EB,2029041,BELT PARKWAY,EB,40.5844494,-73.9273511
138,BELT PARKWAY-WB,2026221,BELT PARKWAY,WB,40.5844494,-73.9273511
308,GRAND CENTRAL PARKWAY-SB,1592929,GRAND CENTRAL PARKWAY,SB,40.7586675,-73.7190057
492,STATEN ISLAND EXPRESSWAY-EB,1509227,STATEN ISLAND EXPRESSWAY,EB,40.6029476,-74.0674953
493,STATEN ISLAND EXPRESSWAY-WB,1429728,STATEN ISLAND EXPRESSWAY,WB,40.6029476,-74.0674953
25,2 AVENUE-SB,1155153,2 AVENUE,SB,40.600834,-74.145898
0,1 AVENUE-NB,1106668,1 AVENUE,NB,40.7704516,-73.9540664
369,MADISON AVENUE-NB,1045637,MADISON AVENUE,NB,40.8120663,-73.9359874


In [400]:

import math 
  
# Function to calculate distance 
def distance(x1 , y1 , x2 , y2): 
  
    # Calculating distance 
    return math.sqrt(math.pow(x2 - x1, 2) +math.pow(y2 - y1, 2) * 1.0) 
  

In [401]:
def min_Dist(lati,long):
    dist=9999999999
    for index,row in nearby_venues.iterrows():
        x1=float(row['lat'])
        y1=float(row['lng'])
        new_dist=distance(x1,y1,float(lati),float(long))
        if(new_dist<dist):
            dist=new_dist
    return dist

In [402]:
new_df["Distance"]=new_df.apply(lambda x: min_Dist(x['Lat'], x['Long']), axis=1)

In [403]:
new_df

,Name_id,Total,Rd,Direction,Lat,Long,Distance
222,CROSS BRONX EXPRESSWAY-EB,2651260,CROSS BRONX EXPRESSWAY,EB,40.8280764,-73.8414339,0.041075
223,CROSS BRONX EXPRESSWAY-WB,2167675,CROSS BRONX EXPRESSWAY,WB,40.8280764,-73.8414339,0.041075
135,BELT PARKWAY-EB,2029041,BELT PARKWAY,EB,40.5844494,-73.9273511,0.011459
138,BELT PARKWAY-WB,2026221,BELT PARKWAY,WB,40.5844494,-73.9273511,0.011459
308,GRAND CENTRAL PARKWAY-SB,1592929,GRAND CENTRAL PARKWAY,SB,40.7586675,-73.7190057,0.034404
492,STATEN ISLAND EXPRESSWAY-EB,1509227,STATEN ISLAND EXPRESSWAY,EB,40.6029476,-74.0674953,0.023135
493,STATEN ISLAND EXPRESSWAY-WB,1429728,STATEN ISLAND EXPRESSWAY,WB,40.6029476,-74.0674953,0.023135
25,2 AVENUE-SB,1155153,2 AVENUE,SB,40.600834,-74.145898,0.037582
0,1 AVENUE-NB,1106668,1 AVENUE,NB,40.7704516,-73.9540664,0.039587
369,MADISON AVENUE-NB,1045637,MADISON AVENUE,NB,40.8120663,-73.9359874,0.048228


In [404]:
nearby_venues.iloc[0]['lat']
new_df['Total'] = new_df['Total']/new_df['Total'].loc[new_df['Total'].abs().idxmax()].astype(np.float64)
new_df['Distance'] = new_df['Distance']/new_df['Distance'].loc[new_df['Distance'].abs().idxmax()].astype(np.float64)

new_df

,Name_id,Total,Rd,Direction,Lat,Long,Distance
222,CROSS BRONX EXPRESSWAY-EB,1.000000,CROSS BRONX EXPRESSWAY,EB,40.8280764,-73.8414339,0.851685
223,CROSS BRONX EXPRESSWAY-WB,0.817602,CROSS BRONX EXPRESSWAY,WB,40.8280764,-73.8414339,0.851685
135,BELT PARKWAY-EB,0.765312,BELT PARKWAY,EB,40.5844494,-73.9273511,0.237591
138,BELT PARKWAY-WB,0.764248,BELT PARKWAY,WB,40.5844494,-73.9273511,0.237591
308,GRAND CENTRAL PARKWAY-SB,0.600820,GRAND CENTRAL PARKWAY,SB,40.7586675,-73.7190057,0.713357
492,STATEN ISLAND EXPRESSWAY-EB,0.569249,STATEN ISLAND EXPRESSWAY,EB,40.6029476,-74.0674953,0.479689
493,STATEN ISLAND EXPRESSWAY-WB,0.539264,STATEN ISLAND EXPRESSWAY,WB,40.6029476,-74.0674953,0.479689
25,2 AVENUE-SB,0.435700,2 AVENUE,SB,40.600834,-74.145898,0.779253
0,1 AVENUE-NB,0.417412,1 AVENUE,NB,40.7704516,-73.9540664,0.820825
369,MADISON AVENUE-NB,0.394392,MADISON AVENUE,NB,40.8120663,-73.9359874,1.000000


In [405]:
#Assigning scores to each of these Locations
new_df["Score"]=new_df["Total"]*new_df["Distance"]

In [406]:
new_df

,Name_id,Total,Rd,Direction,Lat,Long,Distance,Score
222,CROSS BRONX EXPRESSWAY-EB,1.000000,CROSS BRONX EXPRESSWAY,EB,40.8280764,-73.8414339,0.851685,0.851685
223,CROSS BRONX EXPRESSWAY-WB,0.817602,CROSS BRONX EXPRESSWAY,WB,40.8280764,-73.8414339,0.851685,0.696340
135,BELT PARKWAY-EB,0.765312,BELT PARKWAY,EB,40.5844494,-73.9273511,0.237591,0.181831
138,BELT PARKWAY-WB,0.764248,BELT PARKWAY,WB,40.5844494,-73.9273511,0.237591,0.181579
308,GRAND CENTRAL PARKWAY-SB,0.600820,GRAND CENTRAL PARKWAY,SB,40.7586675,-73.7190057,0.713357,0.428599
492,STATEN ISLAND EXPRESSWAY-EB,0.569249,STATEN ISLAND EXPRESSWAY,EB,40.6029476,-74.0674953,0.479689,0.273062
493,STATEN ISLAND EXPRESSWAY-WB,0.539264,STATEN ISLAND EXPRESSWAY,WB,40.6029476,-74.0674953,0.479689,0.258679
25,2 AVENUE-SB,0.435700,2 AVENUE,SB,40.600834,-74.145898,0.779253,0.339520
0,1 AVENUE-NB,0.417412,1 AVENUE,NB,40.7704516,-73.9540664,0.820825,0.342622
369,MADISON AVENUE-NB,0.394392,MADISON AVENUE,NB,40.8120663,-73.9359874,1.000000,0.394392


In [407]:
new_df=new_df.sort_values(by=['Score'], ascending=False)

In [408]:
new_df.index = np.arange(1, len(new_df) + 1)
new_df["Rank"]=new_df.index
new_df

,Name_id,Total,Rd,Direction,Lat,Long,Distance,Score,Rank
1,CROSS BRONX EXPRESSWAY-EB,1.000000,CROSS BRONX EXPRESSWAY,EB,40.8280764,-73.8414339,0.851685,0.851685,1
2,CROSS BRONX EXPRESSWAY-WB,0.817602,CROSS BRONX EXPRESSWAY,WB,40.8280764,-73.8414339,0.851685,0.696340,2
3,GRAND CENTRAL PARKWAY-SB,0.600820,GRAND CENTRAL PARKWAY,SB,40.7586675,-73.7190057,0.713357,0.428599,3
4,MADISON AVENUE-NB,0.394392,MADISON AVENUE,NB,40.8120663,-73.9359874,1.000000,0.394392,4
5,1 AVENUE-NB,0.417412,1 AVENUE,NB,40.7704516,-73.9540664,0.820825,0.342622,5
6,2 AVENUE-SB,0.435700,2 AVENUE,SB,40.600834,-74.145898,0.779253,0.339520,6
7,3 AVENUE-NB,0.391421,3 AVENUE,NB,40.600409,-74.144941,0.785056,0.307288,7
8,STATEN ISLAND EXPRESSWAY-EB,0.569249,STATEN ISLAND EXPRESSWAY,EB,40.6029476,-74.0674953,0.479689,0.273062,8
9,STATEN ISLAND EXPRESSWAY-WB,0.539264,STATEN ISLAND EXPRESSWAY,WB,40.6029476,-74.0674953,0.479689,0.258679,9
10,5 AVENUE-SB,0.370428,5 AVENUE,SB,40.58660115957447,-73.98561242553191,0.687395,0.254630,10


In [411]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()
    #folium.Marker(locationlist[point], popup=df_counters['Name'][point]).add_to(map)
# loop through the 100 crimes and add each to the incidents feature group
for lat, lng,rank in zip(new_df.Lat, new_df.Long,new_df.Rank):
    if(rank<4):
        col='red'
    elif(rank<8):
        col='blue'
    elif(rank<15):
        col='pink'
    else:
        col='green'
    incidents.add_child(
        folium.features.Marker(
            [float(lat), float(lng)],
            #radius=5, # define how big you want the circle markers to be
            #fill=True,
            #fill_color=col,
            #fill_opacity=0.6
           # marker_color='green',
            popup=str(rank),
            icon=folium.Icon(color=col)
        )
    )

# add incidents to map
ny_map.add_child(incidents)